# Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
pd.set_option("display.max_columns", None)

---

# Thu thập dữ liệu

**Dữ liệu được sử dụng trong NoteBook này là thông tin về các khách hàng tham gia cuộc khảo sát để tìm ra khách hàng lý tưởng. Để giúp cho doanh nghiệp hiểu tốt hơn về khách hàng**
> **Context**\
**Problem Statement**\
Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
>
>Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

**Phần mô tả của dữ liệu:**
>**Attributes**\
**People**
>
>- ID: Customer's unique identifier
>- Year_Birth: Customer's birth year
>- Education: Customer's education level
>- Marital_Status: Customer's marital status
>- Income: Customer's yearly household income
>- Kidhome: Number of children in customer's household
>- Teenhome: Number of teenagers in customer's household
>- Dt_Customer: Date of customer's enrollment with the company
>- Recency: Number of days since customer's last purchase
>- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
>
>**Products**
>
>- MntWines: Amount spent on wine in last 2 years
>- MntFruits: Amount spent on fruits in last 2 years
>- MntMeatProducts: Amount spent on meat in last 2 years
>- MntFishProducts: Amount spent on fish in last 2 years
>- MntSweetProducts: Amount spent on sweets in last 2 years
>- MntGoldProds: Amount spent on gold in last 2 years
>
>**Promotion**
>
>- NumDealsPurchases: Number of purchases made with a discount
>- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
>- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
>- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
>- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
>- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
>- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
>
>**Place**
>
>- NumWebPurchases: Number of purchases made through the company’s website
>- NumCatalogPurchases: Number of purchases made using a catalogue
>- NumStorePurchases: Number of purchases made directly in stores
>- NumWebVisitsMonth: Number of visits to company’s website in the last month

**Dữ liệu này được Public trên trang Kaggle nhằm mục đích học tập với license như sau:**
> License CC0: Public Domain


**Cám ơn [LavanyaAkash Patel](https://www.kaggle.com/imakash3011) đã cung cấp tập dữ liệu này**


**Mình download dữ liệu tại đây [kaggle](https://www.kaggle.com/imakash3011/customer-personality-analysis)**

---

# Khám phá dữ liệu

## Đọc dữ liệu từ file .csv

## Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

## Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

## Dữ liệu có các dòng bị lặp không?

## Mỗi cột có ý nghĩa gì?

## Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

---

# Tiền xử lý

---

# Quay lại bước khám phá dữ liệu

## Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

In [32]:
apps.describe()

,Rating,Reviews,Size,Installs,Price
count,8893.000000,1.035800e+04,10358.000000,1.035800e+04,10358.000000
mean,4.189542,4.058657e+05,18.135847,1.415639e+07,1.030701
std,0.545452,2.696651e+06,22.149628,8.023580e+07,16.277843
min,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,4.000000,3.200000e+01,2.700000,1.000000e+03,0.000000
50%,4.300000,1.683000e+03,9.100000,1.000000e+05,0.000000
75%,4.500000,4.640425e+04,26.000000,1.000000e+06,0.000000
max,19.000000,7.815831e+07,100.000000,1.000000e+09,400.000000


ở cột `Rating` có 1 điểm dữ liệu bị sai là `19`, `Rating` cao nhất chỉ là `5` thôi!!!
$\to$ xóa dòng chứa giá trị này.

# Tiền xử lý

Xóa dòng chứa `Rating = 19`

In [45]:
apps.drop(apps[apps.Rating == 19].index, axis=0, inplace=True)

----

# Quay lại bước khám phá dữ liệu

## Với mỗi cột có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

Ở cột `Genres` 1 App có thể có nhiều `Gernes` kết hợp lại và phân tách nhau bởi dấu `;` nên chúng ta cần phải chuyển nó về thành list các **Gerne**.

In [60]:
apps.Genres = apps.Genres.str.split(';')

In [61]:
apps.Genres.head()

0                  [Art & Design]
1    [Art & Design, Pretend Play]
2                  [Art & Design]
3                  [Art & Design]
4      [Art & Design, Creativity]
Name: Genres, dtype: object

**% của mỗi Gerne trong cột `Gernes`**

In [65]:
(apps.Genres.explode().value_counts() / apps.shape[0]) * 100

Tools                      8.139423
Education                  6.922854
Entertainment              6.063532
Business                   4.122815
Medical                    3.939365
Productivity               3.929709
Personalization            3.746259
Lifestyle                  3.611084
Action                     3.582118
Sports                     3.553152
Communication              3.543497
Finance                    3.475910
Photography                3.109008
Health & Fitness           2.973834
Social                     2.703486
News & Magazines           2.549001
Casual                     2.539345
Travel & Local             2.288307
Arcade                     2.259342
Books & Reference          2.249686
Shopping                   2.162788
Simulation                 2.085546
Dating                     1.892440
Video Players & Editors    1.718644
Puzzle                     1.564160
Maps & Navigation          1.322777
Action & Adventure         1.235879
Food & Drink               1

# Đặt câu hỏi 